In [2]:
import pandas as pd 
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize
import keras
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense , Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


In [3]:
TRAIN_DIR = "chest_xray/chest_xray/train/"
TEST_DIR =  "chest_xray/chest_xray/test/"

In [4]:
def set_label(Dir):
    for nextdir in os.listdir(Dir):
        if not nextdir.startswith('.'):
            if nextdir in ['NORMAL']:
                label = 0
            elif nextdir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
    return nextdir, label

In [45]:
def get_data(Dir,type):
    x = []
    y = []
    lenn=0
    if type == 'train':
        for nextDir in os.listdir(Dir):
            if not nextDir.startswith('.'):
                if nextDir in ['NORMAL']:
                    label = 0
                elif nextDir in ['PNEUMONIA']:
                    label = 1
                else:
                    label = 2
                    
                temp = Dir + nextDir
                    
                for file in tqdm(os.listdir(temp)):
                    if lenn<=3000:
                        lenn+=1
                        img = cv2.imread(temp + '/' + file)
                        if img is not None:
                            img = skimage.transform.resize(img, (150, 150, 3))
                            #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                            img = np.asarray(img)
                            x.append(img)
                            y.append(label)
    else:
        
        for nextDir in os.listdir(Dir):
            if not nextDir.startswith('.'):
                if nextDir in ['NORMAL']:
                    label = 0
                elif nextDir in ['PNEUMONIA']:
                    label = 1
                else:
                    label = 2
                   
                temp = Dir + nextDir
                   
                for file in tqdm(os.listdir(temp)):
                    #if lenn<=300:
                      #  lenn+=1
                    img = cv2.imread(temp + '/' + file)
                    if img is not None:
                        img = skimage.transform.resize(img, (150, 150, 3))
                        #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                        img = np.asarray(img)
                        x.append(img)
                        y.append(label)
        
                        
    x = np.asarray(x)
    y = np.asarray(y)
    return x,y

In [46]:
x_train, y_train = get_data(TRAIN_DIR,'train')

100%|██████████| 1342/1342 [00:00<00:00, 1063032.29it/s]


In [34]:
x_test , y_test = get_data(TEST_DIR, 'test')

100%|██████████| 234/234 [01:41<00:00,  2.31it/s]


In [47]:
print(x_train.shape,'\n',x_test.shape)
print(y_train.shape,'\n',y_test.shape)

(3000, 150, 150, 3) 
 (624, 150, 150, 3)
(3000,) 
 (624, 2)


In [48]:
y_train = keras.utils.to_categorical(y_train, 2)
#y_test = keras.utils.to_categorical(y_test, 2)
print(y_train.shape,y_test.shape)

(3000, 2) (624, 2)


In [37]:
#filepath="weights.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [38]:
#K.set_image_data_format('channels_first')
#x_train=x_train.reshape(5216,3,150,150)
#x_test=x_test.reshape(624,3,150,150)

In [76]:
def swish_activation(x):
    return (K.sigmoid(x) * x)

model = Sequential()
# Этот слой превратит нашу двумерную картинку в вектор
model.add(Flatten(input_shape=(3,150,150)))

# Добавляем скрытый полносвязный слой из 128 нейронов
model.add(Dense(units=128, input_shape=(3,150,150)))
#И активацию для скрытого слоя нейронов
model.add(Activation('relu'))
# Добавляем скрытый полносвязный слой из 128 нейронов
model.add(Dense(units=20, input_shape=(3,150,150)))
#И активацию для скрытого слоя нейронов
model.add(Activation('relu'))
# Добавляем скрытый полносвязный слой из 128 нейронов
model.add(Dense(units=20, input_shape=(3,150,150)))
#И активацию для скрытого слоя нейронов
model.add(Activation('relu'))
# Добавляем скрытый полносвязный слой из 128 нейронов
model.add(Dense(units=20, input_shape=(3,150,150)))
#И активацию для скрытого слоя нейронов
model.add(Activation('relu'))

# Добавляем выходной полносвязный слой из 2 нейронов
model.add(Dense(units=2))
# Чтобы получить на выходе вероятности для каждого класса, выбираем активацию
# softmax
#model.add(Activation('softmax'))
#model = Sequential()
#model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(3,150,150)))
#model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#
#model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(3,150,150)))
#model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#
#model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
#model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#
#model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
#model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#
#model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
#model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#
#model.add(Flatten())
#
#model.add(Dense(64, activation=swish_activation))
#model.add(Dropout(0.4))
#model.add(Dense(2 , activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=0.00005),
                  metrics=['accuracy'])

print(model.summary())


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 67500)             0         
_________________________________________________________________
dense_35 (Dense)             (None, 128)               8640128   
_________________________________________________________________
activation_27 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 20)                2580      
_________________________________________________________________
activation_28 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 20)                420       
_________________________________________________________________
activation_29 (Activation)   (None, 20)              

In [77]:
model.fit(x_train, y_train, epochs=6, batch_size=256)

Epoch 1/6
12/12 [==============================] - 4s 293ms/step - loss: 7.0364 - accuracy: 0.9993
Epoch 2/6
12/12 [==============================] - 3s 269ms/step - loss: 7.6246 - accuracy: 1.0000
Epoch 3/6
12/12 [==============================] - 3s 279ms/step - loss: 7.6246 - accuracy: 1.0000
Epoch 4/6
12/12 [==============================] - 3s 278ms/step - loss: 7.6246 - accuracy: 1.0000
Epoch 5/6
12/12 [==============================] - 3s 276ms/step - loss: 7.6246 - accuracy: 1.0000
Epoch 6/6
12/12 [==============================] - 3s 269ms/step - loss: 7.6246 - accuracy: 1.0000


In [78]:
model.evaluate(x_test, y_test)

20/20 [==============================] - 0s 20ms/step - loss: 7.6246 - accuracy: 0.6250


[7.624619483947754, 0.625]

In [79]:
from sklearn.metrics import confusion_matrix
pred = model.predict(x_test)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

In [80]:
CM = confusion_matrix(y_true, pred)
print(CM)

[[  0 234]
 [  0 390]]


In [75]:
390/(390+234)

0.625

In [52]:
model.evaluate(x_train, y_train)

94/94 [==============================] - 1s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]